Install dependencies

In [ ]:
!pip install pandas
!pip install numpy
!pip install jupyter-dash
!pip install matplotlib
!pip install -U kaleido

Import requirements

In [ ]:
from IPython.core.display import display
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import os.path

# Import chart data

Load global charts and track features

In [ ]:
featureData = pd.read_csv('data/track_features.csv')
chartDataGlobal = pd.read_csv('data/chart_global.csv')

Function to import data by region group

In [ ]:
def load_charts_for_codes(codes):
    temp_df = pd.DataFrame()
    for code in codes["code"]:
        if os.path.isfile("data/chart_{}.csv".format(code.lower())):
            temp_df = temp_df.append(pd.read_csv("data/chart_{}.csv".format(code.lower())))
    return temp_df

Load regional chart data

In [ ]:
#Europe
codes = pd.read_csv("data/EuropeCode.csv")
chartDataEurope = load_charts_for_codes(codes)

#Asia
codes = pd.read_csv('data/AsiaCode.csv')
chartDataAsia = load_charts_for_codes(codes)

#NorthAmerica
codes = pd.read_csv('data/NorthAmericaCode.csv')
chartDataNorthAmerica = load_charts_for_codes(codes)

#SouthAmerica
codes = pd.read_csv('data/SouthAmericaCode.csv')
chartDataSouthAmerica = load_charts_for_codes(codes)

#Africa
codes = pd.read_csv('data/AfricaCode.csv')
chartDataAfrica = load_charts_for_codes(codes)

#Oceania
codes = pd.read_csv('data/OcianiaCode.csv')
chartDataOceania = load_charts_for_codes(codes)

# Merging

## prepare data of countries

Function to join chart tracks and track features

In [ ]:
def join_chart_features_regions(chart):
    info = chart.merge(featureData, left_on='track_id', right_on='track_id')
    info = info.groupby(['country', 'date'], as_index=False).mean()
    return info


def join_chart_features_continents(chart):
    info = chart.merge(featureData, left_on='track_id', right_on='track_id')
    info = info.groupby('date', as_index=False).mean()
    return info

Join features and tracks per region and continent

In [ ]:
newDataGlobal = join_chart_features_regions(chartDataGlobal)

newDataEurope = join_chart_features_regions(chartDataEurope)
newDataAsia = join_chart_features_regions(chartDataAsia)
newDataNorthAmerica = join_chart_features_regions(chartDataNorthAmerica)
newDataSouthAmerica = join_chart_features_regions(chartDataSouthAmerica)
newDataAfrica = join_chart_features_regions(chartDataAfrica)
newDataOceania = join_chart_features_regions(chartDataOceania)

meanEurope = join_chart_features_continents(chartDataEurope)
meanAsia = join_chart_features_continents(chartDataAsia)
meanNorthAmerica = join_chart_features_continents(chartDataNorthAmerica)
meanSouthAmerica = join_chart_features_continents(chartDataSouthAmerica)
meanAfrica = join_chart_features_continents(chartDataAfrica)
meanOceania = join_chart_features_continents(chartDataOceania)

In [ ]:
# Average continents
# for trackInfo in chartDataEurope:

# Vis with Plotly

In [ ]:
#danceability	energy	key	loudness	mode	speechiness	acousticness	instrumentalness	liveness	valence	tempo	duration_ms	time_signature

## Graph Helper Functions

In [171]:
def show_line_figure(dataContinent, featureName, continentName):
    newLineFigure = go.Figure()
    newLineFigure.add_trace(go.Scatter(x=dataContinent.index, y=dataContinent[featureName], mode='lines', name='Global'))
    regions = dataContinent['country'].unique()
    for region in regions:
        region_data = dataContinent.loc[dataContinent['country'] == region]
        newLineFigure.add_trace(go.Scatter(x=region_data.date, y=region_data[featureName], mode='lines', name=region))
    newLineFigure.update_layout(
        xaxis_title="Date",
        yaxis_title=featureName,
        legend_title=continentName,
    )
    newLineFigure.show()
    pio.write_image(newLineFigure, "svg/{}-{}.svg".format(featureName, continentName))

def show_line_figure_list(data_dict: {}, featureName):
    newLineFigure = go.Figure()
    for key in data_dict:
        newLineFigure.add_trace(go.Scatter(x=data_dict[key].date, y=data_dict[key][featureName], mode='lines', name=key))
    newLineFigure.update_layout(
        xaxis_title="Date",
        yaxis_title=featureName,
        legend_title="Continents",
    )
    pio.write_image(newLineFigure, "svg/{}.svg".format(featureName))
    newLineFigure.show()


Chart per continent

In [ ]:
continent_set = {
    "Global": newDataGlobal,
    "Europe": meanEurope,
    "Asia": meanAsia,
    "North America": meanNorthAmerica,
    "South America": meanSouthAmerica,
    "Oceania": meanOceania
}

feature_list = [
    "danceability",
    "energy",
    "key",
    "loudness",
    "mode",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "tempo",
    "duration_ms",
    "time_signature"
]

for feature in feature_list:
    show_line_figure_list(continent_set, feature)

Chart per region and feature

In [ ]:
feature_data_set = {
    "Europe": newDataEurope,
    "Asia": newDataAsia,
    "North America": newDataNorthAmerica,
    "South America": newDataSouthAmerica,
    "Oceania": newDataOceania
}

for feature in feature_list:
    for region in feature_data_set:
            show_line_figure(newDataGlobal, feature_data_set[region], feature, region)